In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd

import sys
sys.path.append("../")
import os

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.models import load_LM
import torch

model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    # load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Could not find the bitsandbytes CUDA binary at PosixPath('/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda121.so')
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


AttributeError: /home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cquantize_blockwise_fp16_fp4

In [15]:
from src.dataset import load_worldstate_dataset
dataset = load_worldstate_dataset()
q, a = dataset.__getitem__(5, tags=["a", "b"])
print(q)
print(a)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose the correct option by predicting the answer option after the "Answer:" tag.

Story: Ravi is a farmer in a small Indian village, tending to his mango orchard. Ravi wants to harvest ripe mangoes to sell at the local market. He notices a mango tree with a heavy branch laden with ripe mangoes. A mischievous monkey comes along and eats all the ripe mangoes on the branch while Ravi is gathering his harvesting tools. Ravi sees the monkey eating the ripe mangoes.

Question: Does the branch have ripe mangoes or no ripe mangoes?
a) The branch has ripe mangoes.
b) The branch has no ripe mangoes.
Answer: 
b


In [17]:
from src.models import prepare_input
from src.functional import interpret_logits

inputs = prepare_input(
    prompts = q,
    tokenizer= lm
)

with lm.trace(inputs, validate=False) as tr:
    logits = lm.output.logits.save()

interpret_logits(
    tokenizer=lm,
    logits = logits[:, -1],
    k = 10, get_proba=True
)

[(' __________________', 0.291),
 (' __________________________________', 0.064),
 ('??', 0.056),
 (' ______', 0.055),
 (' (', 0.05),
 ('?', 0.041),
 ('........................', 0.028),
 (' __', 0.026),
 (' **', 0.021),
 (' b', 0.019)]